<a href="https://colab.research.google.com/github/reeshabhkumarranjan/ML-Course-Project/blob/master/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# imports
import numpy as np
import glob
from keras import Sequential
import sklearn, sklearn.model_selection
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense, TimeDistributed, Conv1D, MaxPooling1D, Flatten, ConvLSTM2D, Activation
from keras.utils import to_categorical

In [50]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')

# loading data
colab_directory = '/content/drive/My Drive/Colab Notebooks/ML Course Project/'
x = np.load(colab_directory + "x.npy")
y = np.load(colab_directory + "y.npy")
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.2, random_state=2)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# common variables
epochs = 30
showWorking = False
showWorkingAverage = True
batchSize = 200
dropoutRate = 0.3
activationFunction = 'relu'
iterations = 10

In [52]:
# LSTM model
accuracy_average = 0
for i in range(iterations):
	model_lstm = Sequential([
		LSTM(200, input_shape=(x_train.shape[1], x_train.shape[2])),
		Dropout(dropoutRate),
		Dense(200),
		Activation(activationFunction),
		Dense(y_train.shape[1]),
		Activation('softmax'),
	])
	model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	model_lstm.fit(x_train, y_train, epochs=epochs, batch_size=batchSize, verbose=showWorking)
	accuracy_average += model_lstm.evaluate(x_test, y_test, batch_size=batchSize, verbose=showWorking)[1]
	if showWorkingAverage:
		print(str(i + 1) + ": " + str(accuracy_average / (i + 1)))
accuracy_average /= iterations
print("Accuracy of LSTM model on test: ", accuracy_average)
print()

1: 0.8999999761581421
2: 0.9249999821186066
3: 0.9299999872843424
4: 0.9324999898672104
5: 0.9319999933242797
6: 0.9333333273728689
7: 0.8899999942098346
8: 0.8862499967217445
9: 0.8911111089918349
10: 0.8930000007152558
Accuracy of LSTM model on test:  0.8930000007152558



In [53]:
# CNN-LSTM
accuracy_average = 0
for i in range(iterations):
	trainX = x_train.reshape((x_train.shape[0], 3, 20, x_train.shape[2]))
	testX = x_test.reshape((x_test.shape[0], 3, 20, x_test.shape[2]))

	conv1 = Conv1D(filters=64, kernel_size=5, activation=activationFunction)
	conv2 = Conv1D(filters=64, kernel_size=5, activation=activationFunction)
	model_lstm_cnn = Sequential([
		TimeDistributed(conv1, input_shape=(None, 20, x_train.shape[2])),
		TimeDistributed(conv2),
		TimeDistributed(Dropout(dropoutRate)),
		TimeDistributed(MaxPooling1D(pool_size=2)),
		TimeDistributed(Flatten()),
		LSTM(200),
		Dropout(dropoutRate),
		Dense(200),
		Activation(activationFunction),
		Dense(y_train.shape[1]),
		Activation('softmax')
	])
	model_lstm_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	model_lstm_cnn.fit(trainX, y_train, epochs=epochs, batch_size=batchSize, verbose=showWorking)

	accuracy_average += model_lstm_cnn.evaluate(testX, y_test, batch_size=batchSize, verbose=showWorking)[1]
	if showWorkingAverage:
		print(str(i + 1) + ": " + str(accuracy_average / (i + 1)))
accuracy_average /= iterations
print("Accuracy of LSTM-CNN model on test: ", accuracy_average)
print()

1: 0.949999988079071
2: 0.9449999928474426
3: 0.9399999976158142
4: 0.9399999976158142
5: 0.9360000014305114
6: 0.9316666722297668
7: 0.9300000071525574
8: 0.9287500083446503
9: 0.9311111172040304
10: 0.9330000042915344
Accuracy of LSTM-CNN model on test:  0.9330000042915344



In [54]:
# LSTM + conv
accuracy_average = 0
for i in range(iterations):
	trainX = x_train.reshape((x_train.shape[0], 3, 1, 20, x_train.shape[2]))
	testX = x_test.reshape((x_test.shape[0], 3, 1, 20, x_test.shape[2]))

	model_lstm_conv = Sequential([
		ConvLSTM2D(filters=100, kernel_size=(1,5), input_shape=(3, 1, 20, x_train.shape[2])),
		Activation(activationFunction),
		Dropout(dropoutRate),
		Flatten(),
		Dense(200),
		Activation(activationFunction),
		Dense(y_train.shape[1]),
		Activation('softmax')
	])
	model_lstm_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	model_lstm_conv.fit(trainX, y_train, epochs=epochs, batch_size=batchSize, verbose=showWorking)

	accuracy_average += model_lstm_conv.evaluate(testX, y_test, batch_size=batchSize, verbose=showWorking)[1]
	if showWorkingAverage:
		print(str(i + 1) + ": " + str(accuracy_average / (i + 1)))
accuracy_average /= iterations
print("Accuracy of LSTM-Conv model on test: ", accuracy_average)
print()

1: 0.9399999976158142
2: 0.9300000071525574
3: 0.9300000071525574
4: 0.9300000071525574
5: 0.9300000071525574
6: 0.9300000071525574
7: 0.9300000071525574
8: 0.9300000071525574
9: 0.9300000071525574
10: 0.9300000071525574
Accuracy of LSTM-Conv model on test:  0.9300000071525574

